In [4]:
# Complete tinyGPT
# Then build and train gpt2
# Papers - Attention is all you need
# GPT 2/3 paper

# Llamac/llama2.c
# Cuda 

In [5]:
# how to?
# Model arcthitecture -> 
# Data(find and filter) -> 
# train -> 
# testing/benchmarks -> 
# Sample
# Some other evaluations and researches like scaling laws, transformer sizing
# Configurator
# Play with some opensource model also
# pretraining | post training

In [6]:
from transformers import GPT2LMHeadModel

In [7]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)


transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [8]:
# we train model from as small as one to context length
# so model will see all kinds of context length and from 1 to length of context(block size)
# when predicting the character
# batch - for parallel processing
# In multidimensional matrix
# Each dimension is axis and each axis has index
#dim=0 → collapsing rows → gives a summary per column
# dim=1 → collapsing columns → gives a summary per row
# [a, b, c] = [a][b][c] : same in slicing

In [9]:
# Lets create a BigramNeuralNetwork class with attention in it
vocab_size = 8
n_embd = 2
import torch
import torch.nn as nn
class BigramNeuralNetwork(torch.nn):
    def __init__(self):
        # define how the neural network looks like
        # their structure
        # input layer
        # middle layer
        # output layer
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx):
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        logits = self.lm_head(tok_emb) # (B, T, vocab_size)
    

TypeError: module() takes at most 2 arguments (3 given)

In [ ]:
# Reading Attention is all you need paper
class Block:

    pass

In [ ]:
import torch
B, T, C = 4, 8 , 2
torch.tril(torch.ones(T, T))

NameError: name 'T' is not defined

In [ ]:
from torch.nn import functional as F
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# One attention block
class Head(nn.Module):
    def __init__(self):
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.key = nn.Linear(T, T, bias=False)
        self.query = nn.Linear(T, T, bias=False)
        self.value = nn.Linear(T, T, bias=False)
    
    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        wei = q @ k.tranpose(-1)
        wei = torch.tril(torch.ones(T, T))
        logits = F.softmax(mask, -1)



    pass